In [3]:
# Day 23
#
# https://adventofcode.com/2021/day/23

from copy import *

num_pawn_types = 4

#list of pawn names with pawntype (0-3) as index
P = [chr(ord('A')+p) for p in range(num_pawn_types)]
print("P",P)

#movement cost with pawntype (0-3) as index
C = [10**p for p in range(num_pawn_types)]
print("C",C)

steps = { 'L':(-1,0), 'R':(1,0), 'D':(0,1), 'U':(0,-1)}


######################
#
#  Map operations
#
######################

#Insert agents in map
def addAgents(M, agents):
    m = deepcopy(M)
    for t,x,y,h,v in agents:
        m[y][x] = P[t]
    return m
    

#Render map as text
def render(M):
    t=""
    for l in M:
        t += "".join(l) + "\n"
    return t

def p(M):
    print("".join(M))

######################
#
#  Floodfill distances to prioritize distances to goal
#
######################

def flood(m, depth=0):
    x,y = p
    m[y][x] = str(d)
    for dx,dy in steps.values():
        xx = x+dx
        yy = y+dy
        if m[yy][xx] == ' ':
            m = flood(m, (xx, yy), d+1)
    return m


def make_costmaps(m,t):
    costmaps = []
    for y in reversed(range(len(m))):
        for x in range(len(m[y])):
            if t[y][x]) in range(num_pawn_types):
                print("flood!!!")
                costmaps.append(flood(deepcopy(m), (x,y)))
        print(y)
        if len(costmaps) > 0:
            for i in len(costmaps):
                print("costmap for %c" % (P[i]) )
                p(render(costmaps[i]))
            return costmaps

    
    
######################
#
#  Read inputdata
#
######################

#return clean_map, type_map and start state    
def parse(text):
    #print(text)
    S = []
    M = [list(l) for l in text.split("\n")]
    T = deepcopy(M)
    for y in range(len(M)):
        for x in range(len(M[y])):
            #print(y,x)
            if M[y][x] == ".":
                M[y][x]=" "
                T[y][x]="h"
            
            if M[y][x] in P:
                #create pawn - type, x, y, steps, history, steps
                c=(ord(M[y][x])-ord('A'),x,y,[],[])
                S.append(c)
                
                #mark home tile in type map
                T[y][x] = str(ord(M[y][x])-ord('A'))
                #mark tile as free in clean map
                M[y][x]=" "
                
                #mark nostop zones in type map
                T[1][x] = 'n'
            
            
            
    return M, T, S

######################
#
#  State operations
#
######################

    
#Serialize part of state, so we can recognize situations 
#we have been in before and filter their proposal 
def serialize(state):
    r = []
    state.sort(key=lambda x: int((x[1]+x[2]*100)))
    for t,x,y,h,v in state:
        r.append("%c,%d,%d"%(P[t],x,y))
    return ",".join(r)


######################
#
#  Unfold moves
#
######################


#'i' idle
#' ' regular
#'n' nostop
#'g' goal
#
def legal(agents):
    return True
    for t,x,y,h,v in agents:
        H = "".join(h)
        #V = "".join(v)
        
        #No stopping in front of chambers
        if "n " in H:
            #print("No stopping in front of chambers")
            return False
        
        #Only one stop in hallway allowed
        if H.replace("n", "h").count("h ") > 1:
            #print("Only one stop allowed in hallway")
            return False
        
        #No entry in foreign rooms
        for i in range(num_pawn_types):
            if i != t:
                if "n%d"%(i) in H:
                    #print("No entry in foreign rooms")
                    return False
                
        #no lingering - is taken care of by blacklist
        #for m in ["ud", "du", "lr", "rl"]:
        #    if m in V:
        #        print("Lingering is stupid")
        #        return False
        
        
    return True


def cost(agents, costmaps=None):
    acc=0
    for t,x,y,h,v in agents:
        for z in h:
            if z != ' ':
                #every step counts with the type cost
                acc += C[t]
        if not costmaps is None:
            acc += int(costmaps[t][y][x])*C[t]
    return acc


def isgoal(T,agents):
    #Goal is when all players stand on a home tile of their type
    for t,x,y,h,v in agents:
        if T[y][x] != t:
            return False
    return True



#fold current state int possible 
#new states
def nextstates(M,T,agents):
    
    m=addAgents(M, agents)
    #p(render(T))
    #p(render(m))
    nextstates=[]
    #every agent has a chance
    for i in range(len(agents)):
        t,x,y,h,v=agents[i]
        #print("agent", i, P[t], x, y, h,v)
        
        #Walk through steps
        for stp in steps.keys():
            dx,dy = steps[stp]
            xx=x+dx
            yy=y+dy
            #is move available
            #print("tps:",['#']+P)
            #print("m:", m[yy][xx])
            if m[yy][xx] == ' ':
                move = (t,xx,yy,h + [T[yy][xx]], v+[stp])
                ns = [move]
                for j in range(len(agents)):
                    if i!=j:
                        _t,_x,_y,_h,_v=agents[j]
                        ns.append((_t,_x,_y,_h+[' '], _v+[' ']))
                if legal(ns):
                    nextstates.append(ns)
    return nextstates
           
                    
######################
#
#  Solver
#
######################


def solve(fn, facit, debug=False):
    blacklist={}
    
    M, T, start = parse(open(fn).read())
    
    print("empty map")
    p(render(M))
    
    print("type map")
    p(render(T))
    
    print("costmaps")
    costmaps = make_costmaps(M,T)
        
    
    
    print("starting search")
    e = [(cost(start,costmaps),start)]
    i = 0
    while len(e):
        
        
        c,s = e.pop(0)
        if i % 3000 == 0:
            print("%d - e:%d bl:%d"%(i, len(e), len(blacklist.keys())))
            #print(s)
            p(render(addAgents(M,s)))
            #p(render(T))
            
        if isgoal(T,s):
            p(render(addAgents(M,s)))
        
            print("GOAL!!!")
            print("cost is", cost(s))
            print(s)
            
            
            return
            break
        i += 1
        if i > 100000:
            break
        blacklist[serialize(s)] = 1
        for n in nextstates(M,T,s):
            if not serialize(n) in blacklist:
                e.append((cost(n, costmaps),n))
        
        #sort situations so lowest cost is evaluated first
        e.sort(key=lambda x: x[0])
    
    p(render(addAgents(M,start)))
    print(fn, "no result")
    assert(0)
        
    
solve("i23_test0.txt", None)
solve("i23_test1.txt", None)
solve("i23_test2.txt", None)
solve("i23_test3.txt", None)
solve("i23_test4.txt", None)
solve("i23.txt", 1)

print("fin")


P ['A', 'B', 'C', 'D']
C [1, 10, 100, 1000]
empty map
#######
#     #
## # ##
 #####

type map
#######
#hnhnh#
##1#0##
 #####

costmaps


ValueError: invalid literal for int() with base 10: ' '